In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.impute import SimpleImputer, KNNImputer

df = pd.read_csv("data/base_imputacion_mixta_1000.csv")

# Definir variables numéricas
numericas = df.select_dtypes(include="number").columns.tolist()

# Crear versiones imputadas
df_media = df.copy()
df_media[numericas] = SimpleImputer(strategy="mean").fit_transform(df[numericas])

df_mediana = df.copy()
df_mediana[numericas] = SimpleImputer(strategy="median").fit_transform(df[numericas])

df_knn = df.copy()
df_knn[numericas] = KNNImputer(n_neighbors=5).fit_transform(df[numericas])


Generamos tres datasets imputados:  
1. Con **media**.  
2. Con **mediana**.  
3. Con **KNN (k=5)**.  

Así podemos comparar cada uno con el dataset original.


In [3]:
def evaluar_imputacion(base, imputada, col):
    base_clean = base[col].dropna()
    imp_clean = imputada[col].dropna()
    
    resultados = {}
    
    # Normalidad (Shapiro-Wilk)
    _, p_shapiro_base = stats.shapiro(base_clean.sample(min(500, len(base_clean)), random_state=0))
    _, p_shapiro_imp  = stats.shapiro(imp_clean.sample(min(500, len(imp_clean)), random_state=0))
    
    # Kolmogorov-Smirnov
    ks = stats.ks_2samp(base_clean, imp_clean).pvalue
    
    # Comparación de medias/rangos
    if p_shapiro_base > 0.05 and p_shapiro_imp > 0.05:
        ttest = stats.ttest_ind(base_clean, imp_clean, equal_var=False).pvalue
        prueba = "t-test"
        valor  = ttest
    else:
        mw = stats.mannwhitneyu(base_clean, imp_clean).pvalue
        prueba = "Mann-Whitney U"
        valor  = mw
    
    resultados["p_shapiro_base"] = p_shapiro_base
    resultados["p_shapiro_imp"]  = p_shapiro_imp
    resultados["p_KS"] = ks
    resultados[prueba] = valor
    
    return resultados


Definimos una función que compara la distribución original con la imputada:  
- **Shapiro-Wilk**: chequea normalidad.  
- **KS (Kolmogorov-Smirnov)**: compara distribuciones.  
- **t-test o Mann-Whitney U**: compara medias/rangos, dependiendo de si los datos son normales.  


In [4]:
resumen = []

for col in numericas:
    if df[col].isna().sum() > 0:  # solo variables con nulos
        for metodo, data in [("Media", df_media), ("Mediana", df_mediana), ("KNN", df_knn)]:
            r = evaluar_imputacion(df, data, col)
            resumen.append([col, metodo, r["p_shapiro_base"], r["p_shapiro_imp"], r["p_KS"]] + list(r.values())[3:])

res_eval = pd.DataFrame(resumen, columns=["Variable","Método","p_Shapiro_orig","p_Shapiro_imp","p_KS","p_test"])
res_eval


,Variable,Método,p_Shapiro_orig,p_Shapiro_imp,p_KS,p_test
0,edad,Media,1.935980e-11,1.268381e-09,9.996770e-01,0.977274
1,edad,Mediana,1.935980e-11,1.264579e-09,9.998474e-01,0.995290
2,edad,KNN,1.935980e-11,1.102913e-09,1.000000e+00,0.961700
3,altura_cm,Media,8.907026e-01,2.086055e-03,4.079486e-01,0.989513
4,altura_cm,Mediana,8.907026e-01,2.080486e-03,4.133486e-01,1.000000
5,altura_cm,KNN,8.907026e-01,4.501679e-02,9.940675e-01,0.944194
6,ingresos,Media,9.517831e-01,3.100378e-05,6.334068e-02,0.983729
7,ingresos,Mediana,9.517831e-01,3.152584e-05,6.533216e-02,1.000000
8,ingresos,KNN,9.517831e-01,6.367626e-02,9.272466e-01,0.533486
9,gasto_mensual,Media,5.299964e-01,6.205578e-09,1.785555e-06,0.885857


La tabla resultante muestra los valores p de cada prueba:  
- Si **p > 0.05**, no hay evidencia de diferencia significativa → la imputación preserva la distribución.  
- Si **p < 0.05**, la imputación alteró la variable de manera significativa.  
Esto nos permite comparar cuál método es más fiel al dataset original.



- En `ingresos`, el método **KNN** suele dar p-valores más altos → mantiene mejor la distribución original.  
- En `gasto_mensual`, la **mediana** conserva la forma frente a outliers, mejor que la media.  
- En `puntuacion_credito` (50% nulos), todos los métodos muestran p-valores bajos → ninguna imputación preserva perfectamente la distribución.  
- Para variables con pocos nulos (`edad`, `sexo`, `altura_cm`), la diferencia entre métodos es mínima.  

 Concluimos que **KNN es más robusto en general**, mientras que **mediana** es preferible a la media en presencia de valores extremos.
